# D&A 컨퍼런스

## First  :  Crawling

In [1]:
import selenium 
from selenium import webdriver
import random
from time import sleep
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [2]:
# Browser 켜기
browser = webdriver.Chrome()
browser.get('https://everytime.kr/timetable')

In [3]:
# login 함수 생성
def login(id_,password):
    browser.find_element_by_name('userid').send_keys(str(id_))
    browser.find_element_by_name('password').send_keys(str(password))
    browser.find_element_by_xpath('//*[@id="container"]/form/p[3]/input').click()

In [4]:
login('#','#')

In [6]:
# 가끔 팝업뜨는거 닫아주기
browser.find_element_by_xpath('//*[@id="sheet"]/ul/li[3]/a').click()

In [7]:
browser.find_element_by_xpath('//*[@id="container"]/ul/li[1]').click()

In [8]:
# 스크롤 위치 찾기
scr1=browser.find_element_by_xpath('//*[@id="subjects"]/div[2]')

# 스크롤 계속 내리기 ( 매우 오래걸림 실행 X)
# 하다가 창보고 알아서 끊으면됨
for i in range(100):
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scr1)
    sleep(3)

In [9]:
# 에타 과목 정보 저장
tables = browser.find_elements_by_tag_name("tr")
# 이 과목 평점 저장
stars = browser.find_elements_by_class_name('star')

### 에브리타임 과목정보 크롤링

In [10]:
from tqdm import tqdm_notebook

In [11]:
# 점수랑 타이틀 가져오는 코드
scores=[]
links=[]
for table in tqdm (stars) :
    scores.append(table.get_attribute('title'))
    links.append(table.get_attribute('href'))

100%|██████████████████████████████████████████████████████████████████████████████| 2907/2907 [01:26<00:00, 33.68it/s]


In [12]:
# 테이블이, 3번행부터 정보가 있고, 2918행 이후로는 의미없는 행이길래 제외 (이거 자주 바뀜 확인해줘야 할듯)
# 테이블에서 세부정보 뽑는 코드

tables=tables[3:2909]
grades=[] # 추천학년
categories=[] # 수업 분류(전공,교양)
codes=[] # 과목코드
names=[] # 수업 이름
profs=[] # 교수 이름
weights=[] # 학점(1,2,3)
times=[] # 시간
competitiors=[] # 에타 시간표에 담은인원
remarks=[]


In [13]:
for elem in tqdm_notebook(tables):
    splited=elem.find_elements_by_tag_name('td')
    grades.append(splited[0].text)
    categories.append(splited[1].text)
    codes.append(splited[2].text)
    names.append(splited[3].text)
    profs.append(splited[4].text)
    weights.append(splited[5].text)
    times.append(splited[8].text)
    competitiors.append(splited[10].text)
    remarks.append(splited[11].text)

In [14]:
# 뽑은 리스트들을 df로 바꿔줬음
dfs=[names,codes,scores,categories,profs,times,grades,weights,competitiors,remarks,links]
df=pd.DataFrame()
for cols in dfs:
    df=pd.concat([df,pd.Series(cols)],axis=1)

In [15]:
del tables

In [16]:
df.head()

,0,0,0,0,0,0,0,0,0,0,0
0,역사란무엇인가,002474-01,4.44,핵심교양,윤은주,수 C(12:00-13:15) [북악관4층6호실]\n금 C(12:00-13:15) ...,1-4,3,84,,https://everytime.kr/lecture/view/1415856
1,역사란무엇인가,002474-02,4.44,핵심교양,윤은주,수 D(13:30-14:45) [북악관8층2호실]\n금 D(13:30-14:45) ...,1-4,3,76,,https://everytime.kr/lecture/view/1415856
2,역사란무엇인가,002474-03,0,핵심교양,이시연,화 A(09:00-10:15) [북악관10층13호실]\n목 A(09:00-10:15...,1-4,3,51,,https://everytime.kr/lecture/view/550049
3,한국역사의이해,113240-01,4.28,핵심교양,한준수,수 D(13:30-14:45) [북악관6층5호실]\n금 D(13:30-14:45) ...,1-4,3,60,"국사, 한국역사학과 수강불가",https://everytime.kr/lecture/view/554919
4,한국역사의이해,113240-02,0,핵심교양,김연민,월 A(09:00-10:15) [경상관1층3호실]\n수 A(09:00-10:15) ...,1-4,3,62,"국사, 한국역사학과 수강불가","javascript: alert(""%EC%95%84%EC%A7%81 %EB%93%B..."


In [17]:
# 칼럼 이름 만들어줌
df.columns=['class_name','class_code','score','category','professor','time',
            'recommend_year','weight','competitor','remarks','link']

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2907 entries, 0 to 2906
Data columns (total 11 columns):
class_name        2906 non-null object
class_code        2906 non-null object
score             2907 non-null object
category          2906 non-null object
professor         2906 non-null object
time              2906 non-null object
recommend_year    2906 non-null object
weight            2906 non-null object
competitor        2906 non-null object
remarks           2906 non-null object
link              2907 non-null object
dtypes: object(11)
memory usage: 249.9+ KB


In [19]:
# 아까 위에서 형식 제대로 안지켜진 데이터들 떨굼 ( 보완 방향 (!) )
df.dropna(inplace=True)

In [20]:
df2=df.copy()

In [21]:
df2.to_csv('df2_0926.csv',index=False)

In [22]:
# 링크 추출함, 리뷰 하나도 없는 수업은 java스크립트로 뽑혀오길래 그것도 제외
link_list=list(df2['link'].unique())
link_list.remove('javascript: alert("%EC%95%84%EC%A7%81 %EB%93%B1%EB%A1%9D%EB%90%9C %EA%B0%95%EC%9D%98%ED%8F%89%EC%9D%B4 %EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4.");')

### 에브리타임 리뷰크롤링

In [150]:
from random import randint
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import re
from tqdm import tqdm_notebook

In [151]:
def review_crawl(link_text):
    ## Selenium
    browser.get(link_text)
    sleep(0.5) # 로딩 기다리기
    # 리뷰 내용만 추출
    reg=re.compile('[0-9]')
    reviews={}
    sleep(2) # 로딩 기다리기 
    tb=browser.find_element_by_class_name('articles')
    art_table=tb.find_elements_by_tag_name('article')

    for element in art_table :
        reviews[element.find_element_by_class_name('text').text]=(float(''.join(reg.findall(element.find_element_by_class_name('on').get_attribute('style'))))*0.05)

    # 소스 뺴와서 파싱
    table=bs(browser.page_source,'xml')
    heads=table.find('div',class_='side head')
    
    # 제목, 교수 내용 추출
    class_name=(heads.find('h2').text)
    prof_name=(heads.find('span').text)

    # 강의평 테이블 추출
    articles=table.find('div',class_='side article')

    # 강의평의 평균 평점 내용 추출
    mean_score=float(articles.find('span',class_='value').text)

    # 강의평 중 세부 테이블 추출
    details=articles.find('div',class_='details')

    # 강의평 중 세부 내용 추출
    labels=details.find_all('label')
    detail=details.find_all('span')
    detail_type={}
    for l,d in zip(labels,detail):
        detail_type[str(l.text)]=d.text

    review_t=pd.DataFrame.from_dict(reviews, orient='index').reset_index()
    review_t.columns=['review','score']
    review_t['name']=class_name
    review_t['prof']=prof_name
    
    review_detail=pd.DataFrame.from_dict(detail_type,orient='index').T
    review_detail['name']=class_name
    review_detail['prof']=prof_name

    
    return review_t,review_detail

In [152]:
review=pd.DataFrame() # 강의평 추출 테이블
detail=pd.DataFrame() # 과목 세부정보 추출 테이블

# ↓ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비 
browser = webdriver.Chrome()
browser.get(link_list[0]) 
login('#','#')
# ↑ 가끔 안 켜고 시작하면 오류뜰 때가 있어서 그때 대비 


# 크롤링 과정 (오래걸림 실행 ㄴ)
for link in tqdm_notebook(link_list) :
    try:
        r_t,r_d=review_crawl(link)
        review=pd.concat([review,r_t])
        detail=pd.concat([detail,r_d])
        sleep(1)
    except:
        print(link , '강의평 추출 X ')

https://everytime.kr/lecture/view/550049 강의평 추출 X 
https://everytime.kr/lecture/view/861357 강의평 추출 X 
https://everytime.kr/lecture/view/549879 강의평 추출 X 
https://everytime.kr/lecture/view/1703566 강의평 추출 X 
https://everytime.kr/lecture/view/554824 강의평 추출 X 
https://everytime.kr/lecture/view/861378 강의평 추출 X 
https://everytime.kr/lecture/view/545973 강의평 추출 X 
https://everytime.kr/lecture/view/545979 강의평 추출 X 
https://everytime.kr/lecture/view/1870547 강의평 추출 X 
https://everytime.kr/lecture/view/1950783 강의평 추출 X 
https://everytime.kr/lecture/view/1084644 강의평 추출 X 
https://everytime.kr/lecture/view/1951399 강의평 추출 X 
https://everytime.kr/lecture/view/1416026 강의평 추출 X 
https://everytime.kr/lecture/view/1870555 강의평 추출 X 
https://everytime.kr/lecture/view/550276 강의평 추출 X 
https://everytime.kr/lecture/view/1950800 강의평 추출 X 
https://everytime.kr/lecture/view/1084895 강의평 추출 X 
https://everytime.kr/lecture/view/541701 강의평 추출 X 
https://everytime.kr/lecture/view/541703 강의평 추출 X 
https://everytime.kr/l

https://everytime.kr/lecture/view/1870939 강의평 추출 X 
https://everytime.kr/lecture/view/860225 강의평 추출 X 
https://everytime.kr/lecture/view/1704000 강의평 추출 X 
https://everytime.kr/lecture/view/1870942 강의평 추출 X 
https://everytime.kr/lecture/view/1870943 강의평 추출 X 
https://everytime.kr/lecture/view/1704007 강의평 추출 X 
https://everytime.kr/lecture/view/1870944 강의평 추출 X 
https://everytime.kr/lecture/view/860285 강의평 추출 X 
https://everytime.kr/lecture/view/1419047 강의평 추출 X 
https://everytime.kr/lecture/view/860258 강의평 추출 X 
https://everytime.kr/lecture/view/860288 강의평 추출 X 
https://everytime.kr/lecture/view/1870949 강의평 추출 X 
https://everytime.kr/lecture/view/1704022 강의평 추출 X 
https://everytime.kr/lecture/view/1870950 강의평 추출 X 
https://everytime.kr/lecture/view/1704031 강의평 추출 X 
https://everytime.kr/lecture/view/860318 강의평 추출 X 
https://everytime.kr/lecture/view/860355 강의평 추출 X 
https://everytime.kr/lecture/view/1871154 강의평 추출 X 
https://everytime.kr/lecture/view/548615 강의평 추출 X 
https://everytime.k

In [153]:
#Save
#review.to_csv('review_fixed_0924.csv',index=False)
#detail.to_csv('detail_fixed_0924.csv',index=False)

In [23]:
#Load
review=pd.read_csv('review_fixed_0924.csv')
review=review.reset_index().drop('index',axis=1)
detail=pd.read_csv('detail_fixed_0924.csv')

In [28]:
detail['name'].unique() # good

array(['역사란무엇인가', '한국역사의이해', '임시정부와독립운동', '삶과윤리', '철학의물음들', '시민정신과사회봉사',
       '클래시카', '한국문학의이해', '한문과문화', '음악의이해', '미술의이해', '디자인의이해',
       '공연예술읽기(연극)', '공연예술읽기(무용)', '시민사회와법', '논리와소통', '미디어와공동체', '소통과토론',
       '시민사회와경제', '시민사회와정치', '개인과사회', '회계와사업계획서', '현대사회와여성', '기업가정신과창업',
       '마음과행동', '리더십챌린지', '현대동아시아의이해', '세계문학의이해', '세계역사의이해', '한국사회와문화',
       '문명과세계화의도전', '수학과문명', '생활속의스포츠', '과학과창의적사고', '생명의이해',
       '과학기술과인간사고의변화', '지구생태계와인간', '정보통신과창의적사고', '소프트웨어혁명', '중국어I', '불어',
       '일본어I', '영상을통한영어', '초급중국어회화', '일본어II', '영화독해', '영화작문', '고전과패러디',
       '영미단편소설강독', '영화듣기', '중국어II', '한국어의화용', '기초스페인어', '문화인류학의이해',
       '그리스비극', '동아시아근대사와인물', '예술과철학', '문화권으로보는한국사', '증권투자의이해', '맑시즘의이해',
       '정치전략과전략기획', '교양재테크', '취·창업역량과전략', '취·창업과진로설계', '일본대중문화론',
       '채용시장의이해', '이슬람권문화의이해', '예술,창작과저작권', '영상속일본문화읽기', '지식재산개론',
       '국제관계의이해와통일', '현대문명과재료', '신에너지의생활화', '환경과학의이해', '자동차기술',
       '현대인의식생활과건강', '식생활의문화적이해', '목재의비밀', '숲', '목재와문명', '현대사회와에너지',
       '초급농구', '수영', '탁구', '테니

####  현재 가지고 있는 테이블 
테이블 1: 에브리타임 과목정보  
테이블 2: 에브리타임 리뷰  
테이블 3 :에브리타임 리뷰 요약    

In [29]:
import re
def cleanText(readData):
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', readData) # 특문제거
    text = re.sub('[^가-힣]',' ',text) # 한글 제외, + 단일 자음모음 제거 
    return text

In [30]:
clean=[]
for data in review['review']:
    clean.append(cleanText(data))

In [31]:
review['review']=pd.Series(clean)

----------------------

## Second  :  Text-Mining

## Tokenize 

In [8]:
from konlpy.tag import *
kkma = Kkma()

C:\Users\kcg99\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


 https://m.blog.naver.com/PostView.nhnblogId=wideeyed&logNo=221337575742&proxyReferer=https%3A%2F%2Fwww.google.com%2F

In [172]:
# 토크나이징하고, 에러난 row ( 대부분 빈 텍스트를 토크나이징 한 것 ) 찾아줌
corpus=[] 
error=[]
i=0
for data in tqdm(review['review']):
    try:
        corpus.append(kkma.morphs(data))
    except:
        error.append(i)
    i += 1

100%|████████████████████████████████████████████████████████████████████████████| 11595/11595 [12:38<00:00, 15.29it/s]


In [173]:
error

[2589, 2743, 6065, 6588, 6647, 7793, 8791, 10872]

In [175]:
# 에러난 친구들
display(review.loc[error,:])
review=review.drop(error,0) # 빈 행 삭제

,review,score,name,prof
2589,...,5.0,한국사회와문화,유양석
2743,,2.0,생활속의스포츠,김의진
6065,,1.0,교육학개론,이기종
6588,,5.0,사제동행세미나,이종민
6647,,5.0,광고와사회적이슈,고한준
7793,,1.0,창업연계공학설계입문,김석명
8791,,5.0,컴퓨터구조,임은진
10872,,3.0,동역학,이동헌


In [176]:
import pickle

In [177]:
# save
with open('kkma_token_0924.pickle', 'wb') as f:
    pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

In [32]:
#load
import pickle
with open('kkma_token_0924.pickle', 'rb') as f:
    corpus = pickle.load(f)

_______________ 

## Embedding

In [35]:
from gensim.models import Word2Vec, FastText
embd_SG= Word2Vec(corpus, size=100, window=2, min_count=10,
                           workers=-1, iter=1000, sg=1)
#CBOW보단 Skip - Gram의 성능이 전반적으로 낫다고들 함
#리뷰 여러개니까, 차원 적으면 표현이 안될거라고 생각, 100차원정도 임시로 줘봤음
#리뷰다보니까 별로 안 길어서 윈도우는 작게주고, min_count는 의미없는 단어 나오지 말라고 10정도로 줌
# -> 이거 나중에 보완 ( 2 )

In [36]:
# 리뷰 단위별 벡터구하기 (받은 코드가 옛날버전인듯 word2vec 모델에 그런인자가 없던데..그래서 그냥 짬)
def get_embedding_vector(model,corpus):
    embeding_dict={}
    embeding_df=pd.DataFrame()
    for w,v in zip(model.wv.index2word , model.wv.vectors):
        embeding_dict[w]=v
        embeding_table=(pd.DataFrame(embeding_dict).T.reset_index())
    for words in (corpus):
        embeding_df=pd.concat([embeding_df,
                               embeding_table.query('@words in index').iloc[:,1:].mean()]
                             ,axis=1 )
    return embeding_df

In [37]:
%%time
#임베딩 벡터 추출
embedding_vect=get_embedding_vector(embd_SG,corpus)

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


Wall time: 5min 46s


In [38]:
vect=(embedding_vect.T.reset_index()).iloc[:,1:] # 데이터 프레임으로 바꿔주고
vect.columns=['Review_Embeded'+str(x) for x in range(100)] # 칼럼이름도 달아줌

# 리뷰랑 병합(Group화 해서 강의별 계산을 위함)
review.index=range(len(review))
df3=pd.concat([review.iloc[:,1:],vect],axis=1)

# 여기다 담아놓은다음에 Merge
feature=[]
for col_name in (['Review_Embeded'+str(x) for x in range(100)]):
    feature.append(df3.groupby(['name','prof'])[col_name].mean().reset_index())

In [39]:
#Merge의 과정
df=df2.drop(['class_code','time','link','competitor'],axis=1)
df.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

for f in feature :
    df =pd.merge(df, f, on=['name','prof'],how='left')

df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

In [41]:
sim_df.shape

(966, 102)

In [42]:
# Skip-Gram을 이용한 임베딩 테이블 완성
sim_df=pd.concat([df[['name','prof',]],df.iloc[:,7:]],axis=1)
sim_df.index=[x for x in range(len(sim_df))]

# 나중에 사용할 테이블
detail_table=df2[['class_name','professor','category','recommend_year','weight','score']].drop_duplicates()
detail_table.rename({'class_name':'name','professor':'prof'},axis=1,inplace=True)

In [43]:
sim_df.drop_duplicates(inplace=True)

In [69]:
sim_df.to_csv('sim_df_0926_fixed.csv',index=False)

In [ ]:
"""
df3=pd.concat([df3,pd.get_dummies(df3['category'])],axis=1)
df3=pd.concat([df3,pd.get_dummies(df3['recommend_year'])],axis=1)

df3.drop(['category','recommend_year'],axis=1,inplace=True)

df3=pd.concat([df3[['name','prof','score']],df3.iloc[:,3:]],axis=1)
df3['weight']=df3['weight'].astype(int)

"""


## Similarlity

In [45]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

#코사인 유사도 계산 함수
def cos_sim(A, B): 
       return dot(A, B)/(norm(A)*norm(B))

#다른 모든 행과 유사도 계산 함수
def find_similar(input_name,input_prof,df):
    input_lecture=df.query('name == @input_name & prof == @input_prof')
    rest_lecture=df.query('name != @input_name | prof != @input_prof')
    output=rest_lecture.iloc[:,:2]
    output['similar']=''
    for i in range(len(rest_lecture)):
        output.iloc[i,2]= cos_sim(input_lecture.iloc[:,2:],rest_lecture.iloc[i,2:])
    output = pd.merge(output, detail_table, on=['name','prof'],how='left')
    output = output.sort_values('similar',ascending=False)
    return output

In [47]:
# 내가 들었던 수업중에 교수님이 (사람)좋았던 수업
find_similar('문명과세계화의도전','안현상',sim_df)[:5] # 경제학개론 확인

,name,prof,similar,category,recommend_year,weight,score
12,철학의물음들,이근세,[0.9032072419571805],핵심교양,1-4,3,4
16,시민정신과사회봉사,여현철,[0.9021107026251609],핵심교양,1-4,3,4.2
743,마케팅,이수동,[0.8961569472700733],전공선택,2-4,3,4.02
77,세계문학의이해,방승희,[0.8938402494376912],핵심교양,1-4,3,3.65
93,지구생태계와인간,경종인,[0.8841250204744057],핵심교양,1-4,3,4.32


In [52]:
# 강의는 썩 재미없었지만 교수님이 열정 넘치고 엄청 좋은 분이었음, 유은나 교수님의 경제학 개론도 대체적으로 비슷한 평가
display(review.query('name=="문명과세계화의도전" & prof =="안현상"')[['review']][:5])
display(review.query('name=="철학의물음들" & prof =="이근세"')[['review']][:5]) 

,review
2590,외우기 자신 있고 세계사나 이런거 조금 좋아하면 정말 개꿀일듯 주시는 프린트만...
2591,수업자료 나올거 같은것들 외으면 됨 성적 잘주시는듯 약술형시험인데 그냥 프린트 그...
2592,세계사의 굵직한 흐름들을 스토리텔링으로 알려주셔서 쉽게 맥을 잡을 수 있다 중간 ...
2593,갓갓이라는 말이 맞음 과제업고 조별로 활동하는것도 없고 한학기내내 교수님강의와 ...
2594,암기에 자신있으신 분들 무조건 들으시길 시험 하루전날에 죽고싶은거 빼면 정말 ...


,review
520,저는 이근세교수님이 딱 제 공부스타일에 맞는것 같아요 넘 편해요 다른이근세 교수님 ...
521,갓근세 딱 철학교수님이구나 싶고 철학을 좋아하는 저로서는 흥미로운 수업이었고 ...
522,수업 내용 흥미롭고 재밌어요 시험도 한 번이라 부담 적고 좋습니당 물론 이 한 번...
523,철학에 관심없으면 안듣는게 맞을듯 핵교라 덤볐다가 이번학기 성적 유일하게 안나...
524,무엇보다 가장 큰 장점이자 단점은 시험이 번이라는 겁니다 본인의 노력에 따라 성...


In [54]:
# 외우기만 해서 맘에 안들었던 수업, 영화시리즈 제외하고 한문과 문화 확인
display(find_similar('영화작문','이지현',sim_df)[:5]) 

,name,prof,similar,category,recommend_year,weight,score
108,영화독해,임미진,[0.9498238796068722],자유교양,1-4,2,3.1
113,영화듣기,노윤아,[0.9307878645414617],자유교양,1-4,2,3.8
3,한국역사의이해,남무희,[0.9253001485370783],핵심교양,1-4,3,4
109,영화독해,김혜정,[0.9171173335105252],자유교양,1-4,2,3.68
29,한문과문화,이규일,[0.9097505475413691],핵심교양,1-4,3,4.15


In [55]:
# 암기라는 키워드 자주등장, 차이점은 교수님 칭찬이 많음 ㅎ; 평점차인듯
display(review.query('name=="영화작문" & prof =="이지현"')[['review']][:5])
display(review.query('name=="한국역사의이해" & prof =="남무희"')[['review']][:5]) 


,review
3676,수업시간에 강조하신 문장들을 철저히 암기만 한다면 영어울렁증이 있어도 에이플 가능합...
3677,암기가 중요한 수업입니다 그러나 영화 자체를 반복해서 자주 보고 대사 위주 약간의...
3678,자기가 영어를 잘한다 암기에 문제 없다 그러면 무조건 받아갑니다 년 내...
3679,외국인반 들었는데 엄청 쉬웠어요 한국어를 못해도 시험 잘 볼 듯 명중에서 ...
3680,생각보다 학점따기 어렵습니다 열심히 하는 사람도 많구요 암기 싫어하시는 분 비...


,review
153,독강하기 진짜 좋아요 강의평 보고 시험문제 아무데서나 나올까봐 걱정했는데 교수님이 ...
154,여기 강의평이 모든 걸 말해주고 있다 지엽적인거 씹인정 안녕 내 학점
155,이상한곳에서 문제출제해서 외울게 많다 별 이상한데에서 냄 출석은 끝날때 한번 더 ...
156,팀플없어서 독강하기 완전 조아여 수업도 교수님이 카페에 올려주시는 프린트만 봐도되여...
157,교수님 수업 굉장히 프리하고 좋아요 이번 학기에만 그런건지는 모르겠지만 수업 열심...


## Pre-Trained Embedding

In [56]:
from gensim.models import FastText

In [57]:
%%time
model = FastText.load_fasttext_format('D:/fasttext/wiki.ko.bin')
# Pre-triainde vector를 사용해보고 싶은데, 커뮤니티 리뷰다 보니까 별 이상한 단어랑 줄임말,오타들이 다있음
# -> 그래서 일반적 word2vec모델에선 사용이 안되것 같음
#   -> 근데 FastText는 corpus에 없는 단어들도 사용 가능 + Noise에 강한모델
#      -> 따라서 FastText를 이용한 Pre_Trained Vector 사용 시도

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  """Entry point for launching an IPython kernel.


Wall time: 3min 4s


In [59]:
# save
with open('pre_trained_ft_0926.pickle', 'wb') as f:
    pickle.dump(corpus, f, pickle.HIGHEST_PROTOCOL)

In [58]:
fatx_vec=[] # 문장별 임베딩
for words in corpus:
    fatx_vec.append(model[words].mean(axis=0))

C:\Users\kcg99\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
fatx_emd=pd.DataFrame() # 임베딩한 벡터를 데이터프레임으로 만들어줌
for vecs in (fatx_vec):
    fatx_emd=pd.concat([fatx_emd,pd.DataFrame(vecs)],axis=1)

#벡터 데이터프레임 전처리    
fatx_emd=fatx_emd.T
fatx_emd.index=range(len(fatx_emd))
fatx_emd.columns=['Review_ft_Embeded'+str(x) for x in range(300)]

#그룹바이 하기위해 달아줌
fatx_df=pd.concat([review[['name','prof']],fatx_emd],axis=1)

In [61]:
# 강의별 임베딩 평균값 생성
feature=[]
for col_name in (['Review_ft_Embeded'+str(x) for x in range(300)]):
    feature.append(fatx_df.groupby(['name','prof'])[col_name].mean().reset_index())

In [62]:
# 중복값 제거
fatx_df=fatx_df.iloc[:,:2].drop_duplicates()

In [63]:
# 평균값과 강의명 병합
for f in feature :
    fatx_df =pd.merge(fatx_df, f, on=['name','prof'],how='left')

In [64]:
# 임베딩 테이블 완성
fatx_df.dropna(inplace=True)

In [68]:
fatx_df.to_csv('fatx_df_fixed_0926.csv',index=False)

In [70]:
display(find_similar('역사란무엇인가','윤은주',fatx_df)[:5])
display(find_similar('역사란무엇인가','윤은주',sim_df)[:5])

display(find_similar('영화작문','이지현',fatx_df)[:5])
display(find_similar('영화작문','이지현',sim_df)[:5])
# 결과가 다름 : 수업 분야가 비슷한것들끼리 조금더 잘 뽑은 느낌 (내 주관 + 내가 알고있는 평가)
# 가중치를 줘서 앙상블하는 것도 괜찮지 않을까?

,name,prof,similar,category,recommend_year,weight,score
78,세계역사의이해,윤은주,[0.9987152029380818],핵심교양,1-4,3,4.26
123,맑시즘의이해,김경래,[0.9976613000486264],자유교양,1-4,2,4.14
6,임시정부와독립운동,황선익,[0.9975995692352021],핵심교양,1-4,3,4.19
79,세계역사의이해,고재백,[0.9975762003612858],핵심교양,1-4,3,4.32
824,경영정보학원론,강영신,[0.9975703944332966],전공선택,2-4,3,3.75


,name,prof,similar,category,recommend_year,weight,score
78,세계역사의이해,윤은주,[0.9437045566233755],핵심교양,1-4,3,4.26
134,"예술,창작과저작권",나강,[0.9188773191998691],자유교양,1-4,3,4.7
743,마케팅,이수동,[0.9123551458222875],전공선택,2-4,3,4.02
733,회계학원론,박경재,[0.9050019844017337],전공선택,1,3,4.64
825,경영정보학원론,강영신,[0.8992835789479541],전공선택,2,3,3.75


,name,prof,similar,category,recommend_year,weight,score
108,영화독해,임미진,[0.9986441775744241],자유교양,1-4,2,3.1
113,영화듣기,노윤아,[0.9981812424489236],자유교양,1-4,2,3.8
109,영화독해,김혜정,[0.9978473349327777],자유교양,1-4,2,3.68
58,시민사회와정치,김학량,[0.9977460665214797],핵심교양,1-4,3,3.48
119,동아시아근대사와인물,류미나,[0.9977070367742076],자유교양,1-4,2,3.6


,name,prof,similar,category,recommend_year,weight,score
108,영화독해,임미진,[0.9498238796068722],자유교양,1-4,2,3.1
113,영화듣기,노윤아,[0.9307878645414617],자유교양,1-4,2,3.8
3,한국역사의이해,남무희,[0.9253001485370783],핵심교양,1-4,3,4
109,영화독해,김혜정,[0.9171173335105252],자유교양,1-4,2,3.68
29,한문과문화,이규일,[0.9097505475413691],핵심교양,1-4,3,4.15


### 개선방안
 1) 강의평은 학생이 주관적으로 쓰는것, 게다가 온라인커뮤니티상이라 대충쓰는 경우가 많으며, 에타는 리뷰가 적어서 그게 잘 걸러지지도 않음  
   > 수업계획서를 크롤링해서, 내용이 비슷한것과, 교수님 성향이 비슷한 걸 나눠줄 수 있지 않을까?  
   
   
 2) 머신러닝 모델링을 진행 안했음
   > 이를 이용해 평점에 어떤게 가장 영향을 많이 미치는 지 알 수 있었을텐데 ( 교수 피드백용으로 사용 할 수 있지 않을까?)
   
 3) 디테일 테이블이랑 병합을 할 수 있음
   > 이를 통해 유사도 말고도, 다양한 방법으로 필터링 할 수 있을텐데, 구현해야할 듯

관련 링크:  
 1) https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/  
 2) https://fasttext.cc/docs/en/pretrained-vectors.html  
 3) https://byeongkijeong.github.io/fastText/  
 4) https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/03/11/embedding/
 
 1) word2vec의 전반적 사용법    
 2) fasttext pre-trained-vectors 구한곳  
 3) 그거 사용법 찾은곳   
 4) 임베딩의 전반적 정보 얻은곳

In [ ]:
"""### 전처리 해보려고 했으나 과정이 너무 번거로워서 나중에 Develop 

error_index=df.query('professor in @error_list').index #

error_index.astype(int)

error_table=[]
for i in error_index:
    error_table.append(tables[i].text)

#### 에러의 유형 :
1) 팀팀class의 제목명이 길어서 오류 ( 팀팀class에 대한 분석은 우리task에 별로 맞지 않는듯 ) -> 제거    
2) 강의이름이 영어라 강의 제목에 띄어쓰기가 들어감  
3) 외국인 교수라 교수 이름란에 띄어쓰기가 들어감

import re
error_team= re.compile('팀팀Class')
error_english = re.compile('[a-zA-Z]')

#### 팀팀삭제

error_table2=[]
for table in error_table :
    if len(error_team.findall(table)) == 0 :
        error_table2.append(table)

#### 영어제목 수정

error_eng=[]
error_table3=[]
for table in error_table2:
    if len(error_english.findall(table)) < 5:
        error_table3.append(table)
    else :
        error_eng.append(table)

error_eng[8]

reg_kor=re.compile('[가-힣]')

eng_title=[]
for element in error_eng:
    valid_chr=''.join(element.split(' ')[3:])
    point=(reg_kor.search(valid_chr)).start()
    if len(valid_chr[:point]) >= 1:
        eng_title.append(valid_chr[:point])
    else 

eng_title

### 전처리 해보려고 했으나 과정이 너무 번거로워서 나중에 Develop 
"""